<a href="https://colab.research.google.com/github/chr1shr/am205_g_activities/blob/master/svd_image_processing/compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image compression using SVD

In [ ]:
# start by importing some necessary packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread

We read in a test image and convert to a $[0,1]$ scale before performing any arithmetic.

In [ ]:
im = imread("https://raw.githubusercontent.com/jandrejevic12/svd_files/master/turtle.JPG")
im = im.astype(np.float)/255.

plt.imshow(im)
plt.axis('off')
plt.show()

Next, we vertically stack the color channels to form a $3m\times{n}$ image, where the $n$ columns of the image represent our samples.

In [ ]:
m,n,p = im.shape
S = np.vstack([im[:,:,0],
			         im[:,:,1],
			         im[:,:,2]])

Complete the following function ```factor(S)```, which should take in the data matrix $S$, compute the mean column vector $\bar{S}$ and a centered data matrix $A = S - \bar{S}$, and compute the reduced SVD of $A$. Return the components of the singular value decomposition **and** the computed column mean.

**Note:** ```numpy.linalg.svd``` returns the singular values as a 1D array, not a 2D matrix, and you can leave them in that form.

In [ ]:
def factor(S):


  return U, s, Vt, Sbar # matrix U, array of singular values, matrix V^T, and column mean.

Next, write a function ```rank_approx(U, s, Vt, k)``` which accepts an SVD factorization outputted by ```factor(S)``` and a maximum rank $k$, and assembles a list $L$ of the first $k$ rank-one matrices in the low-rank approximation of the data matrix. The list $L$ should be of the form

$$
L = [\sigma_1 u_1 v_1^T, \sigma_2 u_2 v_2^T, ..., \sigma_k u_k v_k^T]
$$
Return the list as your output.

In [ ]:
def rank_approx(U, s, Vt, k):
    
    
    return L

The function ```reconstruct(L, r, Sbar)``` below accepts the list of low-rank approximations $L$, a target rank $r < k$, and the column mean $\bar{S}$, and returns the rank-$r$ reconstructed image. If your previous functions are set up correctly, you should be able to run the following cell and plot a test reconstruction:

In [ ]:
def reconstruct(L, r, Sbar):
    if r >= len(L):
        print("Error: r must be less than the length of L.")
        return

    m = len(Sbar)//3 # the leading dimension of a single channel

    # Reconstruct the original image up to rank j.
    Ar = np.sum(L[:r], axis=0)
   
    # Add back the mean and reshape into m by n by p.
    Ar += Sbar.reshape(-1,1)
    imr = np.stack([Ar[:m,:],
				            Ar[m:2*m,:],
				            Ar[2*m:,:]], axis=2)
    
    # truncate values and return the reconstructed image.
    imr[imr<0] = 0; imr[imr>1] = 1
    return imr

# Run a test:
k = 120; r = 12
U, s, Vt, Sbar = factor(S)
L = rank_approx(U, s, Vt, k)
imr = reconstruct(L, r, Sbar)

plt.imshow(imr)
plt.axis("off")
plt.show()

If the above test was successful, you should now be able to run the following widget to interactively vary the rank and visualize the resulting reconstruction. With your group, decide how many singular vectors you feel are necessary to reconstruct the image before differences between the original become imperceptible.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive

rmax = 120
U, s, Vt, Sbar = factor(S)
L = rank_approx(U, s, Vt, rmax)

rank_slider = widgets.IntSlider(
    value=1, min=1, max=rmax, step=1,
    description='rank:', continuous_update=False)

def rank_slider_plot(r = 1):
    plt.close()
    
    imr = reconstruct(L, r, Sbar)

    # Plot the original and compressed images.
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,6))
    ax1.imshow(im)
    ax1.set_title("original", size=16)
    ax1.axis("off")

    ax2.imshow(imr)
    ax2.set_title("reconstruction", size=16)
    ax2.axis("off")
    plt.show()


interactive_plot = interactive(rank_slider_plot, r=rank_slider)
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot